In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### To split Test and Train Directories

In [3]:
from sklearn.model_selection import train_test_split
import os
import shutil

# Define paths
original_dataset_dir = 'D:\Btech_AI\Projects\Gesture-recognition\Dataset\images'
base_dir = 'D:\Btech_AI\Projects\Gesture-recognition\Dataset'

# Create base directory
os.makedirs(base_dir, exist_ok=True)

# Define class names
class_names = ['call_me', 'rock_on', 'fingers_crossed', 'okay', 'paper', 'peace', 'rock', 'scissor', 'thumbs', 'up']

# Create train and test directories
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Copy images to train and test directories
for class_name in class_names:
    class_dir = os.path.join(original_dataset_dir, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    train_images, test_images = train_test_split(images, test_size=0.1, random_state=42)
    
    # Copy train images
    train_class_dir = os.path.join(train_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    for img in train_images:
        shutil.copy(img, train_class_dir)
    
    # Copy test images
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(test_class_dir, exist_ok=True)
    for img in test_images:
        shutil.copy(img, test_class_dir)

In [13]:
# Define constants
input_shape = (224, 224, 3)
num_classes = 10
batch_size = 32
epochs = 10

# Data loading and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.10  # Splitting the data into training and validation sets
)

train_generator = train_datagen.flow_from_directory(
    'D:\\Btech_AI\\Projects\\Gesture-recognition\\Dataset\\train',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)


validation_generator = train_datagen.flow_from_directory(
    'D:\\Btech_AI\\Projects\\Gesture-recognition\\Dataset\\train',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 4248 images belonging to 10 classes.
Found 467 images belonging to 10 classes.


### Without K-fold cross Validation

In [14]:
# Model building
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [23]:
# Training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)
# Evaluate the model on the entire training set
train_loss, train_accuracy = model.evaluate(train_generator, verbose=1)

# Evaluate the model on the entire validation set
val_loss, val_accuracy = model.evaluate(validation_generator, verbose=1)

# Print the results
print(f"Training Loss: {train_loss}")
print(f"Training Accuracy: {train_accuracy}")
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/10
132/132 [==============================] - 57s 412ms/step - loss: 0.0499 - accuracy: 0.9839 - val_loss: 0.1084 - val_accuracy: 0.9621
Epoch 2/10
132/132 [==============================] - 54s 411ms/step - loss: 0.0632 - accuracy: 0.9791 - val_loss: 0.2645 - val_accuracy: 0.9174
Epoch 3/10
132/132 [==============================] - 127s 966ms/step - loss: 0.0538 - accuracy: 0.9824 - val_loss: 0.1194 - val_accuracy: 0.9598
Epoch 4/10
132/132 [==============================] - 157s 1s/step - loss: 0.0511 - accuracy: 0.9827 - val_loss: 0.1677 - val_accuracy: 0.9487
Epoch 5/10
132/132 [==============================] - 118s 890ms/step - loss: 0.0339 - accuracy: 0.9893 - val_loss: 0.3094 - val_accuracy: 0.9263
Epoch 6/10
132/132 [==============================] - 59s 438ms/step - loss: 0.0362 - accuracy: 0.9865 - val_loss: 0.1850 - val_accuracy: 0.9487
Epoch 7/10
132/132 [==============================] - 60s 457ms/step - loss: 0.0469 - accuracy: 0.9858 - val_loss: 0.1255 - val_ac

In [20]:
# Save the trained model
model.save('gesture_recognition_model.h5')

C:\Users\Sai Chiranthan H M\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'D:\\Btech_AI\\Projects\\Gesture-recognition\\Dataset\\test',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Set shuffle to False for the testing set
)

Found 1053 images belonging to 10 classes.


In [17]:
# Load the saved model
from tensorflow.keras.models import load_model

model = load_model('gesture_recognition_model.h5')

# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

# Print the accuracy
print("Test Accuracy:", test_accuracy)

33/33 [==============================] - 13s 342ms/step - loss: 0.0707 - accuracy: 0.9782
Test Accuracy: 0.978157639503479


### With K fold cross validation

In [21]:
from sklearn.model_selection import KFold

k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True)

# Lists to store the evaluation results
fold_train_losses = []
fold_train_accuracies = []
fold_val_losses = []
fold_val_accuracies = []

# Iterate over folds
# Perform K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(train_generator)):
    print(f"Fold {fold+1}/{k}")
    ''' 
   # Split data into training and validation sets
    train_generator.reset()  # Reset generator to ensure consistent indexing
    train_data = train_generator.flow_from_directory(
        'D:\\Btech_AI\\Projects\\Gesture-recognition\\Dataset\\train',
        target_size=(input_shape[0], input_shape[1]),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=False,  # Shuffle should be False to maintain consistency with the fold indices
        indices=train_index
    )
    
    val_generator.reset()  # Reset generator to ensure consistent indexing
    val_data = val_generator.flow_from_directory(
        'D:\\Btech_AI\\Projects\\Gesture-recognition\\Dataset\\train',
        target_size=(input_shape[0], input_shape[1]),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False,  # Shuffle should be False to maintain consistency with the fold indices
        indices=val_index
    )
    '''
    # Clone the model
    model_clone = tf.keras.models.clone_model(model)
    model_clone.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model_clone.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size
    )
    
    # Evaluate the model on training set
    train_loss, train_accuracy = model_clone.evaluate(train_generator, verbose=0)
    fold_train_losses.append(train_loss)
    fold_train_accuracies.append(train_accuracy)
    
    # Evaluate the model on validation set
    val_loss, val_accuracy = model_clone.evaluate(validation_generator, verbose=0)
    fold_val_losses.append(val_loss)
    fold_val_accuracies.append(val_accuracy)


# Calculate average metrics
avg_train_loss = np.mean(fold_train_losses)
avg_train_accuracy = np.mean(fold_train_accuracies)
avg_val_loss = np.mean(fold_val_losses)
avg_val_accuracy = np.mean(fold_val_accuracies)

print(f"Average Training Loss: {avg_train_loss}")
print(f"Average Training Accuracy: {avg_train_accuracy}")
print(f"Average Validation Loss: {avg_val_loss}")
print(f"Average Validation Accuracy: {avg_val_accuracy}")


Fold 1/5
Epoch 1/10
132/132 [==============================] - 96s 701ms/step - loss: 2.3010 - accuracy: 0.1105 - val_loss: 2.2969 - val_accuracy: 0.0982
Epoch 2/10
132/132 [==============================] - 92s 700ms/step - loss: 2.2860 - accuracy: 0.1347 - val_loss: 2.2673 - val_accuracy: 0.1830
Epoch 3/10
132/132 [==============================] - 94s 712ms/step - loss: 2.2443 - accuracy: 0.1788 - val_loss: 2.2134 - val_accuracy: 0.1853
Epoch 4/10
132/132 [==============================] - 91s 687ms/step - loss: 2.1825 - accuracy: 0.1895 - val_loss: 2.1624 - val_accuracy: 0.2455
Epoch 5/10
132/132 [==============================] - 89s 675ms/step - loss: 2.1253 - accuracy: 0.1976 - val_loss: 2.1108 - val_accuracy: 0.2411
Epoch 6/10
132/132 [==============================] - 92s 693ms/step - loss: 2.0820 - accuracy: 0.2132 - val_loss: 2.0801 - val_accuracy: 0.2679
Epoch 7/10
132/132 [==============================] - 91s 689ms/step - loss: 2.0569 - accuracy: 0.2125 - val_loss: 2.0616

In [24]:

test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

# Print the test accuracy
print("Test Accuracy: before K-fold cross validation", test_accuracy)

# Evaluate the model on the testing set
test_loss, test_accuracy = model_clone.evaluate(test_generator, verbose=1)

# Print the test accuracy
print("Test Accuracy: After K-fold cross validation", test_accuracy)

33/33 [==============================] - 9s 284ms/step - loss: 0.0264 - accuracy: 0.9905
Test Accuracy: before K-fold cross validation 0.9905033111572266
33/33 [==============================] - 10s 290ms/step - loss: 1.9646 - accuracy: 0.2593
Test Accuracy: After K-fold cross validation 0.25925925374031067
